In [1]:
from threading import Thread

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import io
from scipy.signal import windows
from scipy.spatial import KDTree
from tqdm import tqdm

base_path = 'D:/odrive/grad/traffic_signal/code/ucf_18/images/'
idx_path = 'D:/Dropbox/academic/code/test/benchmark/ucf_18v1_test.csv'

bench_name = 'ucf_18v1'
threshold = 10
conf_score = 0.5


def get_distances(data):
    tree = KDTree(data, leafsize=64)
    distances = tree.query(data, k=2)[0]
    distances = distances[:, 1]

    return distances


def get_kernel(kernlen=49, std=3):
    """Returns a 2D Gaussian kernel array."""
    gkern1d = windows.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d


def gaussian_function(h, w, gt_point, pred_point, sigma):
    point_map = np.zeros((h, w), dtype=np.float32)
    point_map[gt_point[1], gt_point[0]] = 1

    kernel = get_kernel(std=sigma)
    point_map = cv2.filter2D(src=point_map, ddepth=-1, kernel=kernel)
    point_map /= point_map.max()
    val = point_map[pred_point[1], pred_point[0]]
    return val, point_map


def get_variance(h, w, data):
    tree = KDTree(data, leafsize=64)
    gt_idx = tree.query(data, k=2)[1]

    count = 0
    for idx in gt_idx:
        g = data[idx[0]]
        gt_point = g[0], g[1]

        p = data[idx[1]]
        pred_point = p[0], p[1]

        val, _ = gaussian_function(h=h, w=w, gt_point=gt_point, pred_point=pred_point, sigma=threshold)
        # print(val)
        if val >= conf_score:
            count += 1

    return gt_idx.shape[0], count


def validate_gt(np_mat, map_shape):
    gt_list = []
    for val in np_mat:
        x = int(val[0])
        y = int(val[1])
        if (x < map_shape[1]) and (y < map_shape[0]):
            pt = [x, y]
            gt_list.append(pt)

    gt_list = np.array(gt_list)
    return gt_list


def get_gt_list(h, w, mat_file_name):
    if 'shanghai' in bench_name:
        mat = io.loadmat(mat_file_name)
        gt = np.array(mat['image_info'])[0, 0][0, 0][0]
    else:
        if '_4k.' not in img_file_name:
            mat = io.loadmat(mat_file_name)
            gt = np.array(mat['annPoints'])
        else:
            gt = np.load(mat_file_name)

    gt = validate_gt(gt, (h, w))
    # dist = get_distances(gt)
    gt, count = get_variance(h, w, gt)

    # return dist
    return gt, count


def batch_variance(h, w, mat_file_name, i, gt_count):
    gt, count = get_gt_list(h, w, mat_file_name)

    gt_count[i] = [gt, count]


data = pd.read_csv(idx_path)
data['file_name'] = data['file_name'].apply(lambda name: base_path + name)

it = 0
thread_list = []
gt_count = np.zeros((data.size, 2))
dist_list_all = []
for idx, row in data.iterrows():
    img_file_name = row['file_name']
    # if 'v' in bench_name and '_4k.' not in img_file_name:
    #     continue

    if 'test_' not in img_file_name:
        continue

    img = cv2.imread(img_file_name)
    (h, w, c) = img.shape

    if 'shanghai' in bench_name:
        mat_file_name = img_file_name.replace('images', 'ground_truth').replace('IMG', 'GT_IMG').replace('jpg', 'mat')
    else:
        if '_4k.' not in img_file_name:
            mat_file_name = img_file_name.replace('images', 'ground_truth').replace('.jpg', '_ann.mat')
        else:
            mat_file_name = img_file_name.replace('images', 'ground_truth').replace('_4k.jpg', '_ann_4k.npy')

    # print(idx, img_file_name, mat_file_name)

    # gt = get_gt_list(mat_file_name)
    t = Thread(target=batch_variance, args=(h, w, mat_file_name, idx, gt_count))
    t.start()
    thread_list.append(t)

    # dist_list_all += list(gt)

    print(idx)
    it += 1
    if it % 20 == 0:
        for t in thread_list:
            t.join()
    # break

for t in thread_list:
    t.join()

data = pd.DataFrame(gt_count)
data.to_csv('gt_count_' + bench_name + '_' + str(threshold) + '_' + str(conf_score) + '.csv', index=False)

# distances = np.array(dist_list_all)
# plt.hist(distances, range=(0, 100))
# plt.show()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27